<table style="float:left; border:none">
   <tr style="border:none">
       <td style="border:none">  
           <img 
               src="" 
               style="width:10px"
           >
           </a>    
       </td>
   </tr>
</table>
<div style="float:right;"><h2>13. Whole population light gbm classifier OPTUNA</h2></div>

 <a id="top"></a> <br>
## Índice
1. [Entradas y Entregable](#1)<br>
    1.1. [Entradas del modelo](#11)<br>
    1.2. [Entregable](#12)<br>
2. [Carga de paquetes y datos](#2)<br>
    2.1. [Librerías y clases](#21)<br>
    2.2. [Lectura de datos](#22)<br>
    2.3. [Eliminación de valores nulos](#23)<br>
    2.4. [Binarización de variable delta_WHP](#24)<br>
    2.5. [Separación en train y test](#25)<br>
3. [Modelo](#3)<br>
    3.1. [Confección pipeline](#31)<br>
    3.2. [Optimización de hiperparámetros](#32)<br>
    3.3. [Entrenamiento del modelo](#33)<br>
    3.4. [Mejores Hiperparámetros](#34)<br>
4. [Evaluación del modelo](#4)<br>
    4.1. [Cálculo de métricas](#41)<br>
    4.2. [Matriz de confusión](#42)<br>
    4.3. [ROC Curve](#43)<br>
    4.4. [Precision-Recall curve](#44)<br>

***
<a id="1"></a> <br>
## 1. Entradas y Entregable

<a id="11"></a>
### 1.1. Entradas

|Tipo|Nombre|Descripción|Ubicación Lab|
|:-------|----------------|---------------------------------------------------|-------------------------------------|
|**csv**| `Dataset participantes.csv` | datos relacionados con los pozos al producirse una interferencia por la estimulación de un pozo hijo |`/content/drive/MyDrive/DataScience/TRABAJOS PARTICULARES/YPF/DATASET/`|

<a id="12"></a>
### 1.2. Entregable

**Archivo csv con las estimaciones de delta_WHP**

***
<a id="2"></a> <br>
## 2. Carga de paquetes y datos

<a id="21"></a>
### 2.1. Librerías y clases

In [ ]:
! pip install optuna
! pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 5.3 MB/s 
     |████████████████████████████████| 209 kB 49.0 MB/s 
     |████████████████████████████████| 81 kB 9.2 MB/s 
     |████████████████████████████████| 78 kB 7.2 MB/s 
     |████████████████████████████████| 147 kB 66.3 MB/s 
     |████████████████████████████████| 112 kB 54.0 MB/s 
     |████████████████████████████████| 50 kB 6.6 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=4442e0566a283d609b8b17a4d15ced727a3c2a48e7db9db3d4e9ea90010b220a
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 156 kB/s 


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px

import statsmodels.api as sm

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.metrics import recall_score, confusion_matrix, roc_curve, average_precision_score, precision_recall_curve, plot_precision_recall_curve, make_scorer, fbeta_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, binarize
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import lightgbm as lgb
from catboost import CatBoostClassifier
import optuna

import chardet

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Esta clase permite dropear columnas de un DataFrame dentro de un pipeline

class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self,columns):
        self.columns=columns

    def transform(self,X,y=None):
        return X.drop(self.columns,axis=1)

    def fit(self, X, y=None):
        return self  

<a id="22"></a> <br>
### 2.2. Lectura de datos

In [ ]:
# Detección automática de la codificación de los caracteres
with open("/content/drive/MyDrive/DataScience/TRABAJOS PARTICULARES/YPF/DATASET/Dataset participantes (corregido).csv", 'rb') as f:
    enc = chardet.detect(f.read())  # or readline if the file is large

# Lectura del csv
df = pd.read_csv("/content/drive/MyDrive/DataScience/TRABAJOS PARTICULARES/YPF/DATASET/Dataset participantes (corregido).csv",encoding = enc['encoding'], index_col=0)

df.sample(5)

,ID_EVENTO,CAMPO,FLUIDO,PAD_HIJO,HIJO,ETAPA_HIJO,PADRE,D3D,D2D,DZ,AZ,#_BARRERAS,LINEAMIENTO,WHP_i,delta_WHP,ESTADO
ID_FILA,,,,,,,,,,,,,,,,
24553,Pozo 556-Pozo 404-25,Campo E,OIL,PAD-85,Pozo 556,25,Pozo 404,622.616650,620.867376,-46.638977,136.380724,0.0,NO,364.9,0.0,Cerrado
13283,Pozo 247-Pozo 263-25,Campo F,OIL,PAD-80,Pozo 247,25,Pozo 263,1016.077673,1016.042136,-8.497979,249.765940,1.0,NO,198.6,0.0,Abierto
6483,Pozo 243-Pozo 12-10,Campo F,OIL,PAD-80,Pozo 243,10,Pozo 12,1177.797218,1176.801872,48.411162,112.757512,3.0,NO,46.0,0.0,Abierto
21619,Pozo 30-Pozo 62-21,Campo B,OIL,PAD-6,Pozo 30,21,Pozo 62,1069.595731,1068.734546,42.912681,45.641000,0.0,NO,190.0,0.0,Cerrado
22273,Pozo 466-Pozo 29-12,Campo B,OIL,PAD-5,Pozo 466,12,Pozo 29,293.082989,292.838326,-11.973018,266.053501,0.0,NO,233.8,10.2,Cerrado


In [ ]:
# Se cambia el tipo de datos de variables categóricas nominales de object a category
df = df.astype({"CAMPO": "category", "FLUIDO": "category", "PAD_HIJO": "category", "HIJO": "category",
                "ETAPA_HIJO": "category", "PADRE": "category", "LINEAMIENTO": "category", "ESTADO": "category"})
df.dtypes

ID_EVENTO        object
CAMPO          category
FLUIDO         category
PAD_HIJO       category
HIJO           category
ETAPA_HIJO     category
PADRE          category
D3D             float64
D2D             float64
DZ              float64
AZ              float64
#_BARRERAS      float64
LINEAMIENTO    category
WHP_i           float64
delta_WHP       float64
ESTADO         category
dtype: object

<a id="23"></a> <br>
### 2.3. Eliminación de valores nulos
**Dado que el modelo a desarrollar en el presente notebook será tomado como baseline, se eliminan los valores nulos para mayor practicidad**

In [ ]:
print("Cantidad de instancias totales =", df.shape[0])
# Eliminación de todos los valores nulos de dataset
df.dropna(inplace=True)
print("Cantidad de instancias sin valores nulos =", df.shape[0])

Cantidad de instancias totales = 26178
Cantidad de instancias sin valores nulos = 26178


<a id="24"></a> <br>
### 2.4. Binarización de variable delta_WHP

In [ ]:
# Se crea una nueva variable en donde se asigna valor 0 cuando delta_WHP = 0 y valor 1 cuando delta_WHP != 0
df["delta_WHP_binarized"] = df["delta_WHP"].apply(lambda val: 1 if val != 0 else 0)

<a id="24"></a> <br>
### 2.4. Dropeado de variable delta_WHP

In [ ]:
df.drop(['delta_WHP'], axis = 1, inplace = True)

<a id="25"></a> <br>
### 2.5. Separación en train y test

In [ ]:
# Selección de variable target
y = df.delta_WHP_binarized
# Selección de features
X = df.drop(["delta_WHP_binarized"], axis = 1)

# Separación en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25,  stratify= y, random_state = 161)

# Verificación de distribución de variable targuet en train y test
print("Instancias de train \n", X_train.shape[0])
print("Instancias de test \n", X_test.shape[0])

Instancias de train 
 19633
Instancias de test 
 6545


***
<a id="#3"></a> <br>
## 3. Modelo

<a id="31"></a> <br>
### 3.1. Confección pipeline

<a id="32"></a> <br>
### 3.2. Optimización de hiperparámetros

In [ ]:
# Creacion de score f2 
ftwo_scorer = make_scorer(fbeta_score, beta=2)

In [ ]:
# Función objetivo de optuna

def objective_pipe(trial):

  model_type = trial.suggest_categorical('model_type', ['light_gbm_class', 'catboost_class'])

  if model_type == 'light_gbm_class':
    params = {
        'random_state': 48,
        'max_depth': trial.suggest_int('max_depth_lgbm', 5, 50),
        'learning_rate': trial.suggest_categorical('learning_rate_lgbm', [0.001, 0.004, 0.006, 0.008, 0.01, 0.015, 0.018, 0.02]),
        'n_estimators': trial.suggest_int('n_estimators_lgbm', 5, 10),
        'min_data_in_leaf' : trial.suggest_int('min_data_in_leaf_lgbm',10, 300)
    }

    model = lgb.LGBMClassifier(**params)

    
  else model_type == 'catboost_class':

    params = {
        'random_state': 48,
        'learning_rate': trial.suggest_categorical('learning_rate_catboost', [0.001, 0.004, 0.006, 0.008, 0.01, 0.015, 0.018, 0.02]),
        'n_estimators' : trial.suggest_int('n_estimators_catboost', 5, 10),
       'max_depth' : trial.suggest_int('max_depth_catboost', 5, 80),
       'l2_leaf_reg': trial.suggest_int('l2_leaf_reg_catboost', 1, 100)
    }

    model = CatBoostClassifier(**params)

  
  # Instanciado de pipeline
  pipe = Pipeline([ ("ColumnDropper", ColumnDropper(["ID_EVENTO"])),
                    
                    ("column_transformer",ColumnTransformer([
                                          ("standard_scaler", StandardScaler(), 
                      ["D3D", "D2D", "DZ", "AZ", "WHP_i", "#_BARRERAS"])])),
                                  
                                  ("model", model)])
  
   
   
  # Cross validation
  kfold = StratifiedKFold(n_splits=3, random_state=1, shuffle=True)    
    
  # Return
  return cross_val_score(pipe, X_train, y_train, cv= kfold, scoring= ftwo_scorer).mean()


In [ ]:
%%time
models=[]
scores=[]
pipe_study = optuna.create_study(direction='maximize')
pipe_study.optimize(objective_pipe, n_trials=50)
model='light_gbm'
score=pipe_study.best_trial.value
models.append(model)
scores.append(score)

[I 2022-11-07 14:04:31,459] A new study created in memory with name: no-name-7c90ddeb-4407-4e08-965f-5e1050939451
[I 2022-11-07 14:04:34,969] Trial 0 finished with value: 0.0 and parameters: {'model_type': 'light_gbm_class', 'max_depth_lgbm': 10, 'learning_rate_lgbm': 0.02, 'n_estimators_lgbm': 9, 'min_data_in_leaf_lgbm': 151}. Best is trial 0 with value: 0.0.
[I 2022-11-07 14:04:37,445] Trial 1 finished with value: 0.0 and parameters: {'model_type': 'light_gbm_class', 'max_depth_lgbm': 17, 'learning_rate_lgbm': 0.018, 'n_estimators_lgbm': 8, 'min_data_in_leaf_lgbm': 33}. Best is trial 0 with value: 0.0.
[W 2022-11-07 14:04:37,748] Trial 2 failed because of the following error: The value nan is not acceptable.
[I 2022-11-07 14:04:40,038] Trial 3 finished with value: 0.0 and parameters: {'model_type': 'light_gbm_class', 'max_depth_lgbm': 17, 'learning_rate_lgbm': 0.004, 'n_estimators_lgbm': 7, 'min_data_in_leaf_lgbm': 78}. Best is trial 0 with value: 0.0.
[I 2022-11-07 14:04:42,382] Tri

KeyboardInterrupt: ignored

In [ ]:
print('Number of finished trials:', len(pipe_study.trials))
print('Best trial: score {}, params {}'.format(pipe_study.best_trial.value, pipe_study.best_trial.params))

Number of finished trials: 50
Best trial: score 0.6801604447073574, params {'model_type': 'light_gbm_class', 'max_depth_lgbm': 35, 'learning_rate_lgbm': 0.016471109817351864, 'n_estimators_lgbm': 933, 'min_data_in_leaf_lgbm': 129}


In [ ]:
optuna.visualization.plot_optimization_history(pipe_study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(pipe_study)

[W 2022-11-07 13:03:58,699] Your study has only completed trials with missing parameters.


In [ ]:
optuna.visualization.plot_slice(pipe_study)

In [ ]:
optuna.visualization.plot_param_importances(pipe_study)

In [ ]:
optuna.visualization.plot_edf(pipe_study)

In [ ]:
optuna.visualization.plot_contour(pipe_study)

<a id="37"></a> <br>
### 3.7. Entrenamiento del modelo con mejores hiperparámetros

In [ ]:
from sklearn import set_config
set_config(display='diagram')
# with display='diagram', simply use display() to see the diagram
display(pipe)
# if desired, set display back to the default
set_config(display='text')

NameError: ignored

In [ ]:
pipe_study.best_trial.params

{'model_type': 'light_gbm_class',
 'max_depth_lgbm': 35,
 'learning_rate_lgbm': 0.016471109817351864,
 'n_estimators_lgbm': 933,
 'min_data_in_leaf_lgbm': 129}

In [ ]:
model = pipe_study.best_trial.params['model_type']
# Seteado de mejores hiperparametros en el pipeline
if pipe_study.best_trial.params['model_type'] == 'light_gbm_class':
  
  best_params = {
        'random_state': 48,
        'max_depth': pipe_study.best_trial.params['max_depth_lgbm'],
        'learning_rate': pipe_study.best_trial.params['learning_rate_lgbm'],
        'n_estimators': pipe_study.best_trial.params["n_estimators_lgbm"],
        'min_data_in_leaf' : pipe_study.best_trial.params['min_data_in_leaf_lgbm'],
        'num_leaves' : pipe_study.best_trial.params["num_leaves_lgbm"]
             }
  # Instanciado de pipeline lightgbm

  light_gbm_class = lgb.LGBMClassifier(**best_params)
  pipe = Pipeline([ ("ColumnDropper", ColumnDropper(["ID_EVENTO"])),
                    
                    ("column_transformer",ColumnTransformer([
                                          ("standard_scaler", StandardScaler(), 
                      ["D3D", "D2D", "DZ", "AZ", "WHP_i", "#_BARRERAS"])])),
                                  
                                  ("light_gbm_class", light_gbm_class)])


else: 

  best_params = {
        'random_state' : 48,
        'learning_rate' : pipe_study.best_trial.params['learning_rate_catboost'],
        'n_estimators' : pipe_study.best_trial.params["n_estimators_catboost"],
       'max_depth' : pipe_study.best_trial.params['max_depth_catboost'],
       'l2_leaf_reg': pipe_study.best_trial.params['l2_leaf_reg_catboost'],
       'bagging_temperature': pipe_study.best_trial.params['bagging_temperature_catboost']
    }
  # Instanciado de pipeline catboost
  catboost_class = CatBoostClassifier(**best_params)
  pipe = Pipeline([ ("ColumnDropper", ColumnDropper(["ID_EVENTO"])),
                    
                    ("column_transformer",ColumnTransformer([
                                          ("standard_scaler", StandardScaler(), 
                      ["D3D", "D2D", "DZ", "AZ", "WHP_i", "#_BARRERAS"])])),
                                  
                                  ("catboost_class", catboost_class)])


KeyError: ignored

In [ ]:
model = pipe.fit(X_train, y_train)

In [ ]:
from sklearn import set_config
set_config(display='diagram')
# with display='diagram', simply use display() to see the diagram
display(model)
# if desired, set display back to the default
set_config(display='text')


***
<a id="#4"></a> <br>
## 4. Evaluación del modelo

<a id="41"></a> <br>
### 4.1. Cálculo de métricas

In [ ]:
# recall_score
ftwo_score_train = fbeta_score(y_train, model.predict(X_train), beta = 2)
print("F2 train = ", ftwo_score_train)
ftwo_score_test = fbeta_score(y_test, model.predict(X_test), beta = 2)
print("F2 test = ", ftwo_score_test)

<a id="42"></a> <br>
### 4.2. Matriz de confusión

In [ ]:
# Matriz de confusión para modelo
conf_matrix = confusion_matrix(y_train,  model.predict(X_train))
f, ax = plt.subplots(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt="d", linewidths=.6, ax=ax)
plt.title("Confusion Matrix LightGBM Train", fontsize=20)
plt.subplots_adjust(left=0.15, right=0.99, bottom=0.15, top=0.99)
ax.set_yticks(np.arange(conf_matrix.shape[0]) + 0.5, minor=False)
ax.set_xticklabels(['Pred Negatives', 'Pred Positives'],fontsize=16, rotation=360)
ax.set_yticklabels(['Actual Negatives', 'Actual Positives'], fontsize=16, rotation=360)
plt.show()

conf_matrix = confusion_matrix(y_test,  model.predict(X_test))
f, ax = plt.subplots(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt="d", linewidths=.6, ax=ax)
plt.title("Confusion Matrix LightGBM Test", fontsize=20)
plt.subplots_adjust(left=0.15, right=0.99, bottom=0.15, top=0.99)
ax.set_yticks(np.arange(conf_matrix.shape[0]) + 0.5, minor=False)
ax.set_xticklabels(['Pred Negatives', 'Pred Positives'],fontsize=16, rotation=360)
ax.set_yticklabels(['Actual Negatives', 'Actual Positives'], fontsize=16, rotation=360)
plt.show()

<a id="43"></a> <br>
### 4.3. ROC Curve

In [ ]:
# Obtencion de predicciones para graficar curva ROC
probsdmodel = model.predict_proba(X_test)
predsdmodel = probsdmodel[:,1]

lr_false_positive_rate,lr_true_positive_rate,lr_threshold = roc_curve(y_test,predsdmodel)


# Grafico curva ROC
sns.set_style('whitegrid')
plt.figure(figsize=(10,5))
plt.title('Reciver Operating Characterstic Curve')
plt.plot(lr_false_positive_rate,lr_true_positive_rate,label='LightGBM Classifier')

plt.plot([0,1],ls='--')
plt.plot([0,0],[1,0],c='.5')
plt.plot([1,1],c='.5')
plt.ylabel('True positive rate')
plt.xlabel('False positive rate')
plt.legend()
plt.show()

<a id="44"></a> <br>
### 4.4. Precision-Recall Curve

In [ ]:
# Curva Precision-Recall
average_precision = average_precision_score(y_test, predsdmodel)

print(average_precision)

disp = plot_precision_recall_curve(model, X_test, y_test)
disp.ax_.set_title('2-class Precision-Recall curve: '
                   'Average Precision={0:0.2f}'.format(average_precision));

In [ ]:


y_true = y_test
y_pred =  model.predict_proba(X_test)[:,1]

scored_df = pd.DataFrame()
scored_df['target'] = y_test
scored_df['1'] = y_pred
scored_df.index = y_true.index
scored_df.sample(2)

# Define paths de y_true, y_pred_proba
#path_y_true = "tmp/test_set.gz"
y_true_col = "target"

#path_y_pred = "tmp/y_pred_proba_lgbm.csv"
y_pred_col = "1"

#id_col = "cus_cust_id_borrower"

# Define función para unificar target y predicción
def get_scored_df(path_y_true, path_y_pred):
    labels = pd.read_csv(path_y_true, usecols=[id_col, y_true_col])
    preds = pd.read_csv(path_y_pred)
    scored = pd.merge(left=labels, right=preds, left_index=True, right_index=True)
    scored[y_pred_col] = scored[y_pred_col].apply(lambda x: round(x*100, 2))
    return scored

# Armado de dataframe scoreado
#scored_df = get_scored_df(path_y_true, path_y_pred)
scored_df.sort_values(y_pred_col, ascending=False, inplace=True, ignore_index=True)
scored_df

# Reemplaza separador decimal
to_hr = lambda x: "{:,}".format(x).replace(",", ".")

# Preprocesamiento del df scoreado + métricas
pps = [round(x, 3) for x in np.arange(0.001, 1.001, 0.001)]
precisions = list()
recalls = list()
kss = list()
msgs_prec = list()
msgs_recall = list()
msgs_ks = list()
total_clientes = scored_df.shape[0]
total_0s = scored_df[y_true_col].value_counts()[0]
total_1s = scored_df[y_true_col].value_counts()[1]
for pp in pps:
    slice_idx = int(total_clientes * pp)
    current_df = scored_df.loc[:slice_idx-1, :]
    try:
      current_0s = current_df[y_true_col].value_counts()[0]
    except:
      current_0s = 0
    try:
      current_1s = current_df[y_true_col].value_counts()[1]
    except:
      current_1s = 0
    precisions.append(current_1s / slice_idx)
    msgs_prec.append("({} de {})".format(to_hr(current_1s), to_hr(slice_idx)))
    gain_1 = current_1s / total_1s
    recalls.append(gain_1)
    msgs_recall.append("({} de {})".format(to_hr(current_1s), to_hr(total_1s)))
    kss.append(gain_1 - (current_0s / total_0s))

# Genera df con métricas
metrics_df = pd.DataFrame({
    "PP": pps,
    "PRECISION": precisions,
    "RECALL": recalls,
    "KS": kss,
    "msg_PRECISION": msgs_prec,
    "msg_RECALL": msgs_recall})

metrics_df

# Calcula métricas
dfs = list()
for c, new_name in [("PRECISION", "PRECISION"),
                    ("RECALL", "RECALL"),
                    ("KS", "KS")]:
    cdf = metrics_df[["PP", c]].copy()
    cdf.rename(columns={c: "VALOR"}, inplace=True)
    cdf["METRICA"] = new_name
    cdf["msg"] = metrics_df["msg_" + c] if c != "KS" else ""
    dfs.append(cdf)
    
# Unifica df scoreado + métricas para plotear
plot_df = pd.concat(dfs)
for c in ["PP", "VALOR"]:
    plot_df[c] = plot_df[c].apply(lambda x: x*100)

# Plotea
fig = px.line(
    plot_df, x="PP", y="VALOR", color="METRICA",
    width=1000, height=700, text=plot_df["msg"],
    hover_data=["METRICA"])

fig.update_layout(
    hovermode="x",
    font_size=15,
    legend_title_text=None,
    paper_bgcolor="rgba(0, 0, 0, 0)",
    title=dict(text="Modelo Conversión TC MLB - Métricas", font_size=25),
    xaxis=dict(range=[0, 100], title=dict(text="Población (ordenada de mayor a menor score)"), dtick=10),
    yaxis=dict(range=[0, 100], title=dict(text="Valor de la métrica"), dtick=10),
    font=dict(color="black"),
    hoverlabel=dict(font_size=16,))

fig.update_traces(mode="lines", hovertemplate="%{y:.1f}% %{text}")
fig.update_xaxes(showspikes=True, spikecolor="gray", spikesnap="cursor", spikemode="across", ticksuffix="%")
fig.update_yaxes(ticksuffix="%")
fig.update_layout(spikedistance=1000, hoverdistance=100)
fig.show()

In [ ]:
model_predict_proba_train = model.predict_proba(X_train)
model_pred_train = binarize(model_predict_proba_train, threshold = 0.01)[:,1]

model_predict_proba_test = model.predict_proba(X_test)
model_pred_test = binarize(model_predict_proba_test, threshold = 0.01)[:,1]

In [ ]:
# Matriz de confusión para modelo
conf_matrix = confusion_matrix(y_train, model_pred_train)
f, ax = plt.subplots(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt="d", linewidths=.6, ax=ax)
plt.title("Confusion Matrix LightGBM Train", fontsize=20)
plt.subplots_adjust(left=0.15, right=0.99, bottom=0.15, top=0.99)
ax.set_yticks(np.arange(conf_matrix.shape[0]) + 0.5, minor=False)
ax.set_xticklabels(['Pred Negatives', 'Pred Positives'],fontsize=16, rotation=360)
ax.set_yticklabels(['Actual Negatives', 'Actual Positives'], fontsize=16, rotation=360)
plt.show()

conf_matrix = confusion_matrix(y_test, model_pred_test)
f, ax = plt.subplots(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt="d", linewidths=.6, ax=ax)
plt.title("Confusion Matrix LightGBM Test", fontsize=20)
plt.subplots_adjust(left=0.15, right=0.99, bottom=0.15, top=0.99)
ax.set_yticks(np.arange(conf_matrix.shape[0]) + 0.5, minor=False)
ax.set_xticklabels(['Pred Negatives', 'Pred Positives'],fontsize=16, rotation=360)
ax.set_yticklabels(['Actual Negatives', 'Actual Positives'], fontsize=16, rotation=360)
plt.show()